In [63]:
# train data pretreatment
# can't use cross_val to predict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('/home/magic/Documents/train.csv', index_col='Id')
data.shape

y = data.SalePrice
X = data.drop(['SalePrice'], axis=1)

numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
low_object = [col for col in X.columns if X[col].nunique() < 10 and X[col].dtype == 'object']
high_object = [col for col in X.columns if X[col].nunique() > 10 and X[col].dtype == 'object']

from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
imputed_X_num = pd.DataFrame(my_imputer.fit_transform(X[numerical_cols]))
imputed_X_num.columns = X[numerical_cols].columns

from sklearn.preprocessing import LabelEncoder

label_high = X[high_object]

label_encoder = LabelEncoder()
for col in high_object:
    label_high[col] = label_encoder.fit_transform(X[high_object][col])

low = X[low_object]

low['Alley'] = low['Alley'].fillna('None')
low['MasVnrType'] = low['MasVnrType'].fillna('None')
low['BsmtQual'] = low['BsmtQual'].fillna('None')
low['BsmtCond'] = low['BsmtCond'].fillna('None')
low['BsmtExposure'] = low['BsmtExposure'].fillna('None')
low['BsmtFinType1'] = low['BsmtFinType1'].fillna('None')
low['BsmtFinType2'] = low['BsmtFinType2'].fillna('None')
low['Electrical'] = low['Electrical'].fillna('None')
low['FireplaceQu'] = low['FireplaceQu'].fillna('None')
low['GarageType'] = low['GarageType'].fillna('None')
low['GarageFinish'] = low['GarageFinish'].fillna('None')
low['GarageQual'] = low['GarageQual'].fillna('None')
low['GarageCond'] = low['GarageCond'].fillna('None')
low['PoolQC'] = low['PoolQC'].fillna('None')
low['Fence'] = low['Fence'].fillna('None')
low['MiscFeature'] = low['MiscFeature'].fillna('None')


label_encoder = LabelEncoder()
for col in low_object:
    low[col] = label_encoder.fit_transform(low[col])

imputed_X_num = imputed_X_num.reset_index(drop=True)
low = low.reset_index(drop=True)
label_high = label_high.reset_index(drop=True)

total = pd.concat([imputed_X_num, low, label_high], axis=1)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor

X_train, X_valid, y_train, y_valid = train_test_split(total, y, test_size=0.3, random_state=0)

xgb = XGBRegressor(n_estimators=600, learning_rate=0.09)
xgb.fit(X_train, y_train)
prediction = xgb.predict(X_valid)
print("MAE:", mean_absolute_error(prediction, y_valid))
print(prediction)

/home/magic/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/magic/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/magic/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

[12:12:05] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE: 15645.810974957192
[195590.16  150852.16  104539.055 218652.03   90590.81   82623.39
 260384.61  120873.42  388057.53  157087.84  212119.36  147717.27
 233333.48  130647.43  115406.195 150968.02  229946.56  123456.94
 140805.89  204474.05  111522.766 147446.81  107868.21  159834.48
 189812.53  174841.17  170978.4    69183.69  334971.38  116672.38
 122617.69  200448.47  147664.33  293026.28  370581.3   178962.73
 298968.8   121787.78  228570.77  317428.78  210367.3   116374.2
 185759.64  280239.94  351757.6   124612.836 110424.945 133754.62
 170039.64   89626.09  416059.4   147079.06  161687.67   84302.2
 234956.03   95794.38  127086.93  232373.8   129315.96  100198.516
 129579.3   131045.93  133541.16  169249.42  208571.58  149240.5
 140106.94  224876.22  124323.13  193970.67  179057.05  121820.93


In [64]:
# test data pretreatment
# can't use cross_val to predict
test = pd.read_csv('/home/magic/Documents/test.csv', index_col='Id')

numerical_cols = [col for col in test.columns if test[col].dtype in ['int64', 'float64']]
low_object = [col for col in test.columns if test[col].nunique() < 10 and test[col].dtype == 'object']
high_object = [col for col in test.columns if test[col].nunique() > 10 and test[col].dtype == 'object']

from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
imputed_X_test = pd.DataFrame(my_imputer.fit_transform(test[numerical_cols]))
imputed_X_test.columns = test[numerical_cols].columns

high_test = test[high_object]
high_test['Exterior1st'] = high_test['Exterior1st'].fillna('None')
high_test['Exterior2nd'] = high_test['Exterior2nd'].fillna('None')

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for col in high_object:
    high_test[col] = label_encoder.fit_transform(high_test[col])

low_test = test[low_object]

low_test['MSZoning'] = low_test['MSZoning'].fillna('None')
low_test['Alley'] = low_test['Alley'].fillna('None')
low_test['Utilities'] = low_test['Utilities'].fillna('None')
low_test['MasVnrType'] = low_test['MasVnrType'].fillna('None')
low_test['BsmtQual'] = low_test['BsmtQual'].fillna('None')
low_test['BsmtCond'] = low_test['BsmtCond'].fillna('None')
low_test['BsmtExposure'] = low_test['BsmtExposure'].fillna('None')
low_test['BsmtFinType1'] = low_test['BsmtFinType1'].fillna('None')
low_test['BsmtFinType2'] = low_test['BsmtFinType2'].fillna('None')
low_test['KitchenQual'] = low_test['KitchenQual'].fillna('None')
low_test['Functional'] = low_test['Functional'].fillna('None')
low_test['FireplaceQu'] = low_test['FireplaceQu'].fillna('None')
low_test['GarageType'] = low_test['GarageType'].fillna('None')
low_test['GarageFinish'] = low_test['GarageFinish'].fillna('None')
low_test['GarageQual'] = low_test['GarageQual'].fillna('None')
low_test['GarageCond'] = low_test['GarageCond'].fillna('None')
low_test['PoolQC'] = low_test['PoolQC'].fillna('None')
low_test['Fence'] = low_test['Fence'].fillna('None')
low_test['MiscFeature'] = low_test['MiscFeature'].fillna('None')
low_test['SaleType'] = low_test['SaleType'].fillna('None')

label_encoder = LabelEncoder()
for col in low_object:
    low_test[col] = label_encoder.fit_transform(low_test[col])

imputed_X_test = imputed_X_test.reset_index(drop=True)
low_test = low_test.reset_index(drop=True)
high_test = high.reset_index(drop=True)

total_test = pd.concat([imputed_X_test, low_test, high_test], axis=1)
total_test

predicted_prices = xgb.predict(total_test)
print(predicted_prices)

/home/magic/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/magic/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/magic/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

[119885.8  169921.66 180605.83 ... 174209.69 106218.96 217346.8 ]


In [68]:
test = pd.read_csv('/home/magic/Documents/test.csv')
predicted_prices

array([119885.8 , 169921.66, 180605.83, ..., 174209.69, 106218.96,
       217346.8 ], dtype=float32)

In [69]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)